In [0]:
!pip install -U tensorflow

In [0]:
from google.colab import files
import zipfile
import io
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from shutil import copyfile

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [0]:
chk = os.listdir('.')
print(chk)

In [0]:
!rm 'test_subjects3.zip'

In [0]:
uploaded = files.upload()

In [0]:
zpf = zipfile.ZipFile("test_subjects3.zip")
zpf.extractall()
zpf.close()

In [0]:
!mkdir a
!mkdir b
!mkdir c
!mkdir a/images
!mkdir b/images
!mkdir c/images

In [0]:
!rm -r dataset

In [0]:
print(os.listdir('test_subjects3'))

In [0]:
files = os.listdir('test_subjects3')
for f in files:
  cmn = './test_subjects3/'
  if f[-1] == 'g':
    if f[0] == 'a':
      copyfile(cmn + f, './a/images/' + f[1:])
    if f[0] == 'b':
      copyfile(cmn + f, './b/images/' + f[1:])
    if f[0] == 'c':
      copyfile(cmn + f, './c/images/' + f[1:])

In [0]:
print(os.listdir('./a/images'))

In [0]:
table_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)
col_mask_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)
cell_mask_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='constant',
        cval=0)

In [0]:
seed=1
table_generator = table_datagen.flow_from_directory('./a', target_size=(331, 701), class_mode=None, batch_size=30, seed=seed)
col_mask_generator = col_mask_datagen.flow_from_directory('./b', target_size=(331, 701), class_mode=None, batch_size=30, seed=seed)
cell_mask_generator = cell_mask_datagen.flow_from_directory('./c', target_size=(331, 701), class_mode=None, batch_size=30, seed=seed)

In [0]:
#Funkcja z internetu
def new_generator(a,b):
  while True:
    a1 = a.next()
    b1 = b.next()
    yield(a1, b1)

In [0]:
col_train_generator = new_generator(table_generator, col_mask_generator)
cell_train_generator = new_generator(table_generator, cell_mask_generator)

In [0]:
print(col_train_generator)

In [0]:
tab = table_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
tab = col_mask_generator.next()[0]
print(type(tab))
print(len(tab))
print(tab)

In [0]:
#Sprawdzenie transpozycji
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
print(T(np.array([arr])))
#O kurwa jest dobrze :o

In [0]:
#Sprawdzenie reshapea
arr = np.array([
       [[1, 1, 1], [2, 2, 2]],
       [[3, 3, 3], [4, 4, 4]],
       [[5, 5, 5], [6, 6, 6]]
])
print(arr)
T = keras.layers.Permute((2, 1, 3), input_shape=(3, 2, 3))
P = keras.layers.Reshape((2, 9))
w1 = P(T(np.array([arr])))
print(w1)
P2 = keras.layers.Reshape((2, 3, 3))
w2 = P2(w1)
print(w2)
#O kurwa jest dobrze :o

In [0]:
col_model = keras.Sequential([
  keras.layers.Permute((2, 1, 3), input_shape=(331, 701, 3)), #Transpozycja obrazka.
  keras.layers.Reshape((701, 993)), #Pozbycie sie kolorow.
  keras.layers.Dense(101, activation='sigmoid'), #Zmniejszenie wysokosci obrazka.
  keras.layers.Dense(11, activation='sigmoid'), #Zmniejszenie wysokosci obrazka.
  keras.layers.Reshape((701, 11, 1)), #Dodanie kanalu bo warstwa konwolucyjna chce.
  keras.layers.Convolution2D(filters=1, kernel_size=(102, 11), input_shape=(701, 74, 1)), #Przejscie w poziomie.
  keras.layers.Reshape((600, 1)),  #Wywalenie niepotrzebnego wymiaru.
  keras.layers.Convolution1D(filters=1, kernel_size=11), #Przyblizenie konca kolumn. Moze rekurencja?
  keras.layers.Reshape([590]), #Splaszczenie.
  keras.layers.Dense(701, activation='hard_sigmoid'), #Lokalizacja kolumn i przywrócenie orginalnej szerokosci.
  keras.layers.Reshape((701, 1)), #Dodanie wymiaru.
  keras.layers.Dense(993, activation='linear'), #Odtworzenie wysokosci.
  keras.layers.Reshape((701, 331, 3)), #Przywrócenie kanałow.
  keras.layers.Permute((2, 1, 3), input_shape=(701, 331, 3)) #Ponowna transpozycja.
])

In [0]:
col_model.compile(loss='mae', optimizer='adam')

In [0]:
his = col_model.fit_generator(
    col_train_generator,
    use_multiprocessing=True,
    steps_per_epoch=30,
    epochs=20,
    validation_data=col_train_generator,
    validation_steps=30)

In [0]:
tab = table_generator.next()[0]

In [0]:
array_to_img(tab)

In [0]:
prd = col_model.predict(np.array([tab]))

In [0]:
print(prd[0])

In [0]:
array_to_img((prd[0])) #Siec generuje poziome paski... Czyli coś skopalaem z tymi transpozycjami. Teraz poszukam co dokladnie i dodam detekcje
#faktycznej tabelki.